<a href="https://colab.research.google.com/github/asandy1023/Assembly/blob/main/langchain_llama_cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-cpp-python

In [ ]:
!pip install numpy
!pip install sentencepiece==0.1.98

In [ ]:
!pip install langchain

In [ ]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
!git lfs install
!git clone https://huggingface.co/hlhr202/llama-7B-ggml-int4

In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/content/llama-7B-ggml-int4/ggml-model-q4_0.bin", callback_manager=callback_manager, verbose=True, n_ctx=1024
)

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

llm_chain.run(question)

# part 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install chromadb
!pip install sentence_transformers

In [ ]:
import openpyxl

# 讀取Excel檔案
workbook = openpyxl.load_workbook('/content/drive/Shareddrives/nnn/langchain/data/A1_new_客服審核 1-3.xlsx')

## 選擇工作表
worksheet = workbook.active

# 創建一個空字串來存放結果
result_max = ''
result_min = ''
result_list = []
i_max = 0
i_min =0
# 遍歷每一行
for row in worksheet.iter_rows(min_row=1, max_row=3, min_col=3, max_col=4):
    # 取得C欄與D欄的值，並加入結果字串中
    c_value = row[0].value if row[0].value else ''
    d_value = row[1].value if row[1].value else ''

    c_list = c_value.split("\n")
    i_max = 0
    for value in c_list:
        if value !='':
            result_max = f'{value}:{d_value}'
            result_list.append(result_max)

print(result_list)

['台灣大哥大職缺查詢:台灣大哥大列入「公司治理評鑑前5%績優公司」，菁英人才職缺招募:https://hrweb.taiwanmobile.com/TWMHRMS/forms/JobType.aspx，\n對於程式開發/資料庫管理/雲端平台服務有興趣，HIPO疾行軍:https://hrweb.taiwanmobile.com/TWMHRMS/forms/hipo02.aspx\n想成為最夯 APP 軟體工程師\n攻城精英就業保證班立即報名:https://hrweb.taiwanmobile.com/TWMHRMS/forms/i\n2023/2/17 23:59截止報名\n2023/2/08 18:30線上說明會:https://www.accupass.com/event/2212290254311091217186\n影像連結：https://hrweb.taiwanmobile.com/TWMHRMS/forms/welf01.aspx', 'HIPO疾行軍職缺跟培訓:台灣大哥大列入「公司治理評鑑前5%績優公司」，菁英人才職缺招募:https://hrweb.taiwanmobile.com/TWMHRMS/forms/JobType.aspx，\n對於程式開發/資料庫管理/雲端平台服務有興趣，HIPO疾行軍:https://hrweb.taiwanmobile.com/TWMHRMS/forms/hipo02.aspx\n想成為最夯 APP 軟體工程師\n攻城精英就業保證班立即報名:https://hrweb.taiwanmobile.com/TWMHRMS/forms/i\n2023/2/17 23:59截止報名\n2023/2/08 18:30線上說明會:https://www.accupass.com/event/2212290254311091217186\n影像連結：https://hrweb.taiwanmobile.com/TWMHRMS/forms/welf01.aspx', 'AppWorks School就業保證班:台灣大哥大列入「公司治理評鑑前5%績優公司」，菁英人才職缺招募:https://hrweb.taiwanmobile.com/TWMHRMS/forms/JobType.aspx，\n對於程式開發/資料庫管理/雲端平台服

In [ ]:
result_list[11]

'我想申請Disney+:申辦Disney+請留下資料:請提供方便聯絡的電話，專人會在上班時間第1個工作天內與您聯繫說明～上班時間：週一至週五 1000-1900，國定例假日休。限網內門號洽詢，未成年/外籍人士/公司戶須前往門市辦理哦！\n另外當您提供個人相關資料，代表已知悉並同意本公司隱私權聲明內容。\n隱私權聲明連結：https://www.taiwanmobile.com/footer/static-privacy.html?utm_source=twm_footer&utm_medium=footer&utm_campaign=privacy'

In [ ]:
len(result_list)

12

In [ ]:
ids=[]
metadatas=[]
for idx, x in enumerate(result_list):
  ids.append("id"+str(idx))
  metadatas.append({"source": "my_source"})

In [ ]:
ids

In [ ]:
metadatas

In [ ]:
import chromadb
chroma_client = chromadb.Client()

collection = chroma_client.create_collection(name="my_collection")
collection.delete()
collection.add(
    documents=result_list,
    metadatas=metadatas,
    ids=ids
)

In [ ]:
results = collection.query(
    query_texts=["申請Disney+"],
    n_results=1
)

In [ ]:
results

{'ids': [['id10']],
 'embeddings': None,
 'documents': [['我想了解Disney+:申辦Disney+請留下資料:請提供方便聯絡的電話，專人會在上班時間第1個工作天內與您聯繫說明～上班時間：週一至週五 1000-1900，國定例假日休。限網內門號洽詢，未成年/外籍人士/公司戶須前往門市辦理哦！\n另外當您提供個人相關資料，代表已知悉並同意本公司隱私權聲明內容。\n隱私權聲明連結：https://www.taiwanmobile.com/footer/static-privacy.html?utm_source=twm_footer&utm_medium=footer&utm_campaign=privacy']],
 'metadatas': [[{'source': 'my_source'}]],
 'distances': [[1.0192179679870605]]}

#3

In [ ]:
!git lfs install
!git clone https://huggingface.co/hlhr202/llama-7B-ggml-int4 /content/drive/Shareddrives/nnn/langchain

In [ ]:
!pip install llama-cpp-python
!pip install unstructured
!pip install pytesseract
!pip install numpy==1.24
!pip install sentencepiece==0.1.98
!pip install langchain

In [ ]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/content/drive/Shareddrives/nnn/langchain/ggml-model-q4_0.bin", 
    callback_manager=callback_manager, 
    verbose=True, 
    n_ctx=2048
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI,VectorDBQA
from langchain.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA

# 加载文件夹中的所有txt类型的文件
loader = DirectoryLoader('/content/drive/Shareddrives/nnn/langchain/data', glob='**/*.txt')
# 将数据转成 document 对象，每个文件会作为一个 document
documents = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
documents

[Document(page_content='台灣大哥大職缺查詢,HIPO疾行軍職缺跟培訓,AppWorks School ,台灣大哥大列入「公司治理評鑑前5%績優公司」，菁英人才職缺招募:https://hrweb.taiwanmobile.com/TWMHRMS/forms/JobType.aspx，\n\n對於程式開發/資料庫管理/雲端平台服務有興趣，HIPO疾行軍:https://hrweb.taiwanmobile.com/TWMHRMS/forms/hipo02.aspx\n\n想成為最夯 APP 軟體工程師攻城精英就業保證班立即報名:https://hrweb.taiwanmobile.com/TWMHRMS/forms/i2023/2/17 23:59截止報名\n\n2023/2/08 18:30線上說明會:https://www.accupass.com/event/2212290254311091217186 影像連結：https://hrweb.taiwanmobile.com/TWMHRMS/forms/welf01.aspx', metadata={'source': '/content/drive/Shareddrives/nnn/langchain/data/test2.txt'})]

In [ ]:
# 初始化加载器
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# 切割加载的 document
split_docs = text_splitter.split_documents(documents)

In [ ]:
split_docs

[Document(page_content='台灣大哥大職缺查詢,HIPO疾行軍職缺跟培訓,AppWorks School ,台灣大哥大列入「公司治理評鑑前5%績優公司」，菁英人才職缺招募:https://hrweb.taiwanmobile.com/TWMHRMS/forms/JobType.aspx，\n\n對於程式開發/資料庫管理/雲端平台服務有興趣，HIPO疾行軍:https://hrweb.taiwanmobile.com/TWMHRMS/forms/hipo02.aspx\n\n想成為最夯 APP 軟體工程師攻城精英就業保證班立即報名:https://hrweb.taiwanmobile.com/TWMHRMS/forms/i2023/2/17 23:59截止報名\n\n2023/2/08 18:30線上說明會:https://www.accupass.com/event/2212290254311091217186 影像連結：https://hrweb.taiwanmobile.com/TWMHRMS/forms/welf01.aspx', metadata={'source': '/content/drive/Shareddrives/nnn/langchain/data/test2.txt'})]

In [ ]:
from langchain.embeddings import LlamaCppEmbeddings
llama = LlamaCppEmbeddings(model_path="/content/drive/Shareddrives/nnn/langchain/ggml-model-q4_0.bin")

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [ ]:
from langchain.vectorstores import Chroma
# 初始化 openai 的 embeddings 对象
#embeddings = OpenAIEmbeddings()
# 将 document 通过 openai 的 embeddings 对象计算 embedding 向量信息并临时存入 Chroma 向量数据库，用于后续匹配查询
docsearch = Chroma.from_documents(split_docs, llama)

In [ ]:
docsearch

In [ ]:
text = "台灣大哥大職缺查詢"

query_result = llama.embed_query(text)

doc_result = llama.embed_documents([text])

In [ ]:
query_result

In [ ]:
from langchain.chains import RetrievalQA
# create qa target vector check embedding weight
qa = VectorDBQA.from_chain_type(llm=llm, chain_type="stuff", vectorstore=docsearch,return_source_documents=True)
# start qa
result = qa({"query": "HIPO疾行軍職缺"})
print(result)

ERROR:langchain.vectorstores.chroma:Chroma collection langchain contains fewer than 4 elements.
ERROR:langchain.vectorstores.chroma:Chroma collection langchain contains fewer than 3 elements.
ERROR:langchain.vectorstores.chroma:Chroma collection langchain contains fewer than 2 elements.
Llama.generate: prefix-match hit


In [ ]:
result = qa({"query": "台灣大哥大職缺查詢"})
print(result)